In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

class results:
    def __init__(self):
        S = 0.0
        R2 = 0.0
        IKA = 0.0

In [28]:
#math

def white_noise(N = 1):
    white_noise_vector = np.zeros((N,1))
    for i in range(N):
        white_noise_vector[i] = np.random.normal(0,1,size=None)    
    return white_noise_vector

def ARMA(AR, MA, coefficients, y_amount, v = None):
    if v is None:
        v = white_noise(y_amount)
    y = np.zeros((y_amount,1))
    for i in range(max(AR, MA)):
        y[i] = v[i]
    X = np.zeros((y_amount - max(AR, MA), AR + MA + 2))
    for i in range(max(AR, MA),y_amount):
        X[i-max(AR, MA)][0] = 1
        for j in range(1,AR+1):
            X[i-max(AR, MA)][j] = y[i-j]
        for j in range(AR+1, AR + MA + 2):
            X[i-max(AR, MA)][j] = v[i - j + AR + 1]
        
        y[i] = X[i-max(AR, MA)] @ coefficients.T
    return y, X

def less_square_method(X, Y, real_coefficients):
    coefficients_predict = np.linalg.inv(X.T @ X) @ X.T @ Y
    error_vector = coefficients_predict-real_coefficients
    standard_deviation = np.sum(np.power(error_vector, 2))
    return standard_deviation, error_vector

def recursive_less_square_method(X, Y, real_coefficients, a = 0.01, b = 10):
    i = 0
    sigma_past = np.zeros((X.shape[1], 1))
    P_past = b * np.diag(np.ones(X.shape[1]))
    for i in range(X.shape[0]):
        row = np.array([X[i]])
        P_current = P_past - (P_past * row.T * row * P_past)/(1 + row * P_past * row.T)
        sigma_current = sigma_past + P_current @ row.T * (Y[i].T - row @ sigma_past)
        P_past = P_current
        sigma_past = sigma_current
    error_vector = sigma_current-real_coefficients
    standard_deviation = np.sum(np.power(error_vector,2))
    return standard_deviation, error_vector

def coefficient_of_determination(RSS, vector_for_TSS):
    TSS = np.sum(np.power(vector_for_TSS-np.mean(vector_for_TSS),2))
    return 1-RSS/TSS

def IKA(e, AR, MA):
    sum = np.sum(np.power(e,2))
    return e.size * np.log(sum) + 2 * (AR + MA + 1)

In [ ]:
#interface 1

error_text = ""

def read_var_from_file(file_dir):
    try:
        file = open(file_dir, 'r')
    except FileNotFoundError:
        error_text = "File not found"
        return False, error_text, None
    var_str = file.readlines()
    amount_of_var = len(var_str)
    if(amount_of_var == 0):
        error_text = "File is empty"
        return False, error_text, None
    var = np.zeros((amount_of_var,1))
    for i in range(len(var_str)):
        try:
            var[i] = float(var_str[i])
        except ValueError:
            error_text = "Data in file is not readable"
            return False, error_text, None
    file.close()
    return True, amount_of_var, var

def coefficients_string(coefficients):
    string = ''
    for element in coefficients:
        string += str(element) + ' '
    return string

def input_AR_coef(AR, AR_coefficients):
    result = np.zeros((AR + 1,1))
    for i in range(AR+1):
        try:
            result[i] = float(input("a%d = " % (i)))
        except ValueError:
            return False, AR_coefficients
    return True, result

def input_MA_coef(MA, MA_coefficients):
    result = np.zeros((MA,1))
    for i in range(MA):
        try:
            result[i] = float(input("b%d = " % (i+1)))
        except ValueError:
            return False, MA_coefficients
    return True, result

def input_coef(AR, MA, coefficients):
    success, AR_coefficients = input_AR_coef(AR, None)
    if not success:
        return False, coefficients
    success, MA_coefficients = input_MA_coef(MA, None)
    if not success:
        return False, coefficients
    return True, make_coefficients(AR_coefficients, MA_coefficients)

def make_coefficients(AR, MA):
    result = np.zeros((len(AR)+len(MA)+1,1))
    for i in range(len(AR)):
        result[i] = AR[i]
    result[len(AR)] = 1
    for i in range(len(MA)):
        result[i+len(AR)+1] = MA[i]
    return result

def input_y_amount(number_of_y):
    result = 0
    try:
        result = int(input("number of y = "))
    except ValueError:
        return False, number_of_y
    return True, result

def input_dir(string):
    try: 
        return True, input(string)
    except:
        return False, None

def show_menu(taked_data_from_file, AR, MA, coefficients, number_of_y, some_text_about_errors = ''):
    print(  "Configurate ARMA\n"+
            "1. Autoregression : %d\n" % (AR) +
            "2. Moving avarage : %d\n" % (MA) +
            "3. Coefficients : %s\n" % (coefficients_string(coefficients)) +
            "4. Amount of y : %s\n" % (str(number_of_y) if not taked_data_from_file else str(number_of_y) + "lock") +
            "5. Taked v from file : %s\n" % ("yes lock" if taked_data_from_file else "no") +
            "6. End configuration\n" + 
            some_text_about_errors)
    return input()

def menu():
    taked_data_from_file = False
    v = None
    AR, MA = 0, 0
    AR_coefficients, MA_coefficients = np.ones(AR + 1), np.ones(MA)
    coefficients = np.ones(AR + MA + 2)
    number_of_y = 10
    error_text = ''
    while(True):
        os.system('cls')
        change = show_menu(taked_data_from_file,AR,MA,coefficients,number_of_y, error_text)
        os.system('cls')
        error_text = ""
        try:
            change = int(change)
        except ValueError:
            error_text = "Invalid input. Type 1-6 number"
        if change not in [1,2,3,4,5,6]:
            error_text = "Invalid input. Type 1-6 number"
            continue
        if change == 4 and taked_data_from_file:
            print("You can`t change amount of y if you taked data from file")
        if change == 5 and taked_data_from_file:
            print("You want discard data taked from file?(y/n)")
            change = input()
            if change == 'y':
                print("Discarded")
                taked_data_from_file = False
                change = 4
            elif change == 'n':
                change = 7
            else:
                error_text = "Invalid input"
        if change == 1:
            try:
                data = int(input("New AR amount : "))
            except ValueError:
                error_text = "Invalid input"
                continue
            if data > 100 or data < 0:
                error_text = "Invalid input"
                continue
            success, AR_coefficients = input_AR_coef(data, AR_coefficients)
            if not success:
                error_text = "Invalid input. Changes discarded"
                continue
            AR = data
            coefficients = np.zeros((AR+MA+2,1))
            coefficients = make_coefficients(AR_coefficients, MA_coefficients)            
        if change == 2:
            try:
                data = int(input("New MA amount : "))
            except ValueError:
                error_text = "Invalid input"
                continue
            if data > 100 or data < 0:
                error_text = "Invalid input"
                continue
            success, MA_coefficients = input_MA_coef(data, MA_coefficients)
            if not success:
                error_text = "Invalid input. Changes discarded"
                continue
            MA = data
            coefficients = np.zeros((AR+MA+2,1))
            coefficients = make_coefficients(AR_coefficients, MA_coefficients)  
        if change == 3:
            coefficients = np.zeros((AR+MA+2,1))
            success, coefficients = input_coef(AR, MA, coefficients)
            if not success:
                error_text = "Invalid input. Changes discarded"
                continue
        if change == 4:
            success, number_of_y = input_y_amount(number_of_y)
            if not success:
                error_text = "Invalid input. Changes discarded"
                continue
            if data <= 0:
                error_text = "Invalid input"
                continue
            if data <= max(AR, MA) + 1:
                error_text = "Invalid input. Amount of y can`t be lower that AR + 1 or MA + 1"
                continue
        if change == 5:
            num = number_of_y
            success, dir = input_dir("Input directory to file : ")
            if not success:
                error_text = "Unknown error"
                continue
            success, number_of_y, v = read_var_from_file(dir)
            if not success:
                error_text = number_of_y
                v = None
                number_of_y = num
                continue
            else:
                taked_data_from_file = True
        if change == 6:
            result = ARMA(AR, MA, np.array([coefficients]), number_of_y, v)
            return result[0], result[1], coefficients, AR, MA

def results_showing(AR, MA, LSM, RLSM):
    print(  "Result for ARMA(%d,%d):\n" % (AR,MA) +
            "LSM:\n" +
            "S = %f\n" % (LSM.S) +
            "R^2 = %f\n" % (LSM.R2) +
            "IKA = %f\n" % (LSM.IKA) +
            "RLSM:\n" +
            "S = %f\n" % (RLSM.S) +
            "R^2 = %f\n" % (RLSM.R2) +
            "IKA = %f\n" % (RLSM.IKA))

def main():
    y, X, real_coefficients, AR, MA = menu()
    print("Vector y:")
    print(y)
    Y = y[max(AR, MA):]
    LSM, RLSM = results(), results()
    LSM.S, LSM.e = less_square_method(X, Y, real_coefficients)
    RLSM.S, RLSM.e = recursive_less_square_method(X, Y, real_coefficients)
    LSM.R2 = coefficient_of_determination(LSM.S, real_coefficients)
    RLSM.R2 = coefficient_of_determination(RLSM.S, real_coefficients)
    LSM.IKA = IKA(LSM.e, AR, MA)
    RLSM.IKA = IKA(RLSM.e, AR, MA)
    results_showing(AR, MA, LSM, RLSM)

In [29]:
#interface 2
def rand_coefficients(AR, MA):
    result = np.zeros((1,AR+MA+2))
    sum = 0.01
    for i in range(AR+MA+2):
        result[0][i] = np.random.random()
        sum += abs(result[0][i])
    for i in range(AR+MA+2):
        result[0][i] /= sum
    result[0][AR + 1] = 1
    return result

def results_showing(AR, MA, LSM, RLSM):
    print(  "Result for ARMA(%d,%d):\n" % (AR,MA) +
            "LSM:\n" +
            "S = %f\n" % (LSM.S) +
            "R^2 = %f\n" % (LSM.R2) +
            "IKA = %f\n" % (LSM.IKA) +
            "RLSM:\n" +
            "S = %f\n" % (RLSM.S) +
            "R^2 = %f\n" % (RLSM.R2) +
            "IKA = %f\n" % (RLSM.IKA))
            
def main():
    for AR in range(1,4):
        for MA in range(1,4):
            coefficients = coefficients(AR,MA)
            y, X = ARMA(AR,MA,coefficients, 200)
            Y = y[max(AR, MA):]
            LSM, RLSM = results(), results()
            LSM.S, LSM.e = less_square_method(X, Y, coefficients)
            RLSM.S, RLSM.e = recursive_less_square_method(X, Y, coefficients)
            LSM.R2 = coefficient_of_determination(LSM.S, coefficients)
            RLSM.R2 = coefficient_of_determination(RLSM.S, coefficients)
            LSM.IKA = IKA(LSM.e, AR, MA)
            RLSM.IKA = IKA(RLSM.e, AR, MA)
            results_showing(AR, MA, LSM, RLSM)

In [31]:
main()

Result for ARMA(1,1):
LSM:
S = 4.206008
R^2 = -7.000000
IKA = 28.984225
RLSM:
S = 4.015066
R^2 = -6.636819
IKA = 28.240860

Result for ARMA(1,2):
LSM:
S = 5.640322
R^2 = -9.000000
IKA = 51.248530
RLSM:
S = 5.476692
R^2 = -8.709893
IKA = 50.512533

Result for ARMA(1,3):
LSM:
S = 7.943879
R^2 = -11.000000
IKA = 84.606459
RLSM:
S = 7.890759
R^2 = -10.919758
IKA = 84.364925

Result for ARMA(2,1):
LSM:
S = 6.414085
R^2 = -9.000000
IKA = 54.462409
RLSM:
S = 6.797801
R^2 = -9.598239
IKA = 55.914978

Result for ARMA(2,2):
LSM:
S = 8.505514
R^2 = -11.000000
IKA = 87.065728
RLSM:
S = 8.544823
R^2 = -11.055459
IKA = 87.231723

Result for ARMA(2,3):
LSM:
S = 8.865659
R^2 = -13.000000
IKA = 118.927079
RLSM:
S = 9.804654
R^2 = -14.482792
IKA = 123.860001

Result for ARMA(3,1):
LSM:
S = 6.806867
R^2 = -11.000000
IKA = 79.045549
RLSM:
S = 690.566935
R^2 = -1216.418177
IKA = 245.350465

Result for ARMA(3,2):
LSM:
S = 9.416970
R^2 = -13.000000
IKA = 121.883156
RLSM:
S = 9.682513
R^2 = -13.394777
IKA = 1

In [33]:
y = np.array([0.655390, 0.324468, 0.454957, 0.235509, 0.313953, 0.254530, 0.251973, 0.235613])
y = y.T
X = np.array([  [0.200904, 0.655390, 0.324468, 0.454957, 0.235509, 0.313953, 0.254530, 0.251973],
                [1.063189, 0.200904, 0.655390, 0.324468, 0.454957, 0.235509, 0.313953, 0.254530]])
X = X.T
np.linalg.inv(X.T @ X) @ X.T @ y

array([0.28531016, 0.55569261])